[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/accelerate_pytorch_inference_jit_ipex.ipynb)

# Accelerate PyTorch Inference in Precision Bfloat16


> 📝 **Note**
>
> * `bf16`: All inference optimizations in bfloat16 precision should use the `InferenceOptimizer.quantize(..., precision='bf16')`.
> * `bf16 + ipex`: You can use `InferenceOptimizer.quantize(..., precision='bf16', use_ipex=True)` API to enable the ipex acceleration for PyTorch inference.
> * `bf16 + jit`: You can use `InferenceOptimizer.quantize(..., precision='bf16', accelerator="jit")` API to enable the jit acceleration for PyTorch inference.
> * `bf16 + channels_last`: You can use `InferenceOptimizer.quantize(..., precision='bf16', channels_last=True)` to use channels last memory format for PyTorch inference.

To accelerate the model in bf16 precision, the following dependencies need to be installed first：

In [ ]:
# for BigDL-Nano
!pip install --pre --upgrade bigdl-nano[pytorch]  # install the nightly-bulit version
# !source bigdl-nano-init


> 📝 **Note**
>
> We recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

Let's take an [ResNet-18 model](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) pretrained on ImageNet dataset as an example. First, we load the model:

In [ ]:
import torch
from torchvision.models import resnet18

model_ft = resnet18(pretrained=True)


Then we set it in evaluation mode:

In [ ]:
model_ft.eval()


Accelerate the model in bf16 precision, we need import `InferenceOptimizer` firstly.

In [ ]:
from bigdl.nano.pytorch import InferenceOptimizer


### Inference with bfloat16

In [ ]:
x = torch.rand(2, 3, 224, 224)
bf16_model = InferenceOptimizer.quantize(model_ft,
                                            precision='bf16',
                                            input_sample=torch.rand(1, 3, 224, 224))
with InferenceOptimizer.get_context(bf16_model):
    y_hat = bf16_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)


### Accelerate inference using IPEX optimizer in bf16 precision

In [ ]:
ipex_model = InferenceOptimizer.quantize(model_ft,
                                         precision='bf16',
                                         use_ipex=True,
                                         input_sample=torch.rand(1, 3, 224, 224))
with InferenceOptimizer.get_context(ipex_model):
    y_hat = ipex_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)



### Accelerate inference using JIT in bf16 precision

In [ ]:
jit_model = InferenceOptimizer.quantize(model_ft,
                                        precision='bf16',
                                        accelerator="jit",
                                        input_sample=torch.rand(1, 3, 224, 224))
with InferenceOptimizer.get_context(jit_model):
    y_hat = jit_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)


### Accelerate inference using channels last memory format in bf16 precision

In [ ]:
jit_model = InferenceOptimizer.quantize(model_ft,
                                        precision='bf16',
                                        channels_last=True,
                                        input_sample=torch.rand(1, 3, 224, 224))
with InferenceOptimizer.get_context(jit_model):
    y_hat = jit_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)


### Accelereate inference using combined method
You can use any of the methods mentioned above to combine each other to try to acclerate your model. However, the effect is not like stacking buffs.
It is not that the more methods you use, the better. You should try many times to find the best combination of methods.

In [ ]:
# bf16 + IPEX + JIT + channels_last
jit_ipex_model = InferenceOptimizer.quantize(model_ft,
                                             precision='bf16',
                                             accelerator="jit",
                                             use_ipex=True,
                                             channels_last=True,
                                             input_sample=torch.rand(1, 3, 224, 224))
with InferenceOptimizer.get_context(jit_ipex_model):
    y_hat = jit_ipex_model(x)
    predictions = y_hat.argmax(dim=1)
    print(predictions)


> 📝 **Note**
>
> `input_sample` is the parameter for OpenVINO accelerator to know the **shape** of the model input. So both the batch size and the specific values are not important to `input_sample`. If we want our test dataset to consist of images with $224 \times 224$ pixels, we could use `torch.rand(1, 3, 224, 224)` for `input_sample` here.
>
> Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/pytorch.html#bigdl.nano.pytorch.InferenceOptimizer.quantize) for more information on `InferenceOptimizer.quantize`.

> 📚 **Related Readings**
>
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
> - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)